In [26]:
import tensorflow.compat.v1 as tf
from tensorflow import feature_column
from tensorflow.keras import layers
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [27]:
seed = 0
# a = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))
# b = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))

# tensorflow, numpy 랜덤 값을 설정합니다.
tf.set_random_seed(seed)
np.random.seed(seed)

In [28]:
#doginfo.csv파일 데이터를 pandas를 이용해 읽어옵니다.
dog_data=pd.read_csv("doginfo.csv")
dog_train=pd.read_csv("trainDoginfo.csv")
kindCd=pd.read_csv("kindCd.csv")

dog_data = dog_data.dropna(axis=0)
dog_train = dog_train.dropna(axis=0)
kindCd_data = kindCd.dropna(axis=0)

dog_data
# print(dog_train)
# print(kindCd)

,kindNum,neuterYn,sexCd,weight,noticeDays,age2,processState
0,128.0,N,F,7.46,10,12,0
1,114.0,N,M,7.00,14,1,1
2,114.0,U,M,4.50,11,2,0
3,67.0,N,M,10.00,8,1,0
4,114.0,N,M,6.00,8,4,0
...,...,...,...,...,...,...,...
22787,114.0,N,M,1.00,10,0,1
22788,114.0,N,M,1.00,10,0,1
22789,114.0,N,M,1.00,10,0,1
22790,128.0,U,F,6.00,12,3,0


In [29]:
kindCd = np.array(kindCd_data, dtype = np.float64)
kindCd_train=np.array(kindCd_data, dtype=np.float64)

kindCd = kindCd.reshape(177)
kindCd_train=kindCd_train.reshape(177)
print(kindCd)
print(kindCd.shape)

[ 54.  56.  55. 118. 115.  37.  81. 204.  83.  82.  38.  39.  40.  43.
  42. 153.  41. 120. 155.  69.  71. 142.  93. 167.  70. 166.  94. 121.
 152.  73. 146.  72. 159.  76.  75.  79.  78.  77.  74.  80. 114. 133.
  12.  17.  15. 164. 157. 148.  16.  20.  21.  22.  24. 208.  23.  26.
  27. 169.  25.  19.  13.  18.  14. 162.  85.  96.  95.   1.  34. 104.
  31.  99. 122. 123.  97. 132. 105. 154. 124. 100. 103. 151. 139. 101.
 102.  98. 136. 202. 160. 203.   8. 131.   9. 119. 150. 210.  57.  58.
  59.   6.   4.   7.   5. 143.  11.  10. 137.  84. 163. 112. 113. 149.
 211. 110. 205. 108. 109.  60.  46.  47.  44.  45.  53.  62.  61.  52.
 165.  51. 156. 129.  67.  35.  33.  32. 158. 144.  30.  29.  64. 207.
  28.   2.  68. 125. 141. 145.  36.  66.  65.  63. 140. 107. 106. 209.
  86.  88.  90.  87. 138.  89. 126. 127. 128.  91.   3. 161.  50. 168.
  49. 147.  92.  48. 135. 206. 130. 134. 111.]
(177,)


In [30]:
# kindNum을 원핫 인코딩
kindCd = pd.concat((pd.get_dummies(dog_data.kindNum, columns=kindCd), pd.DataFrame(columns=kindCd))).fillna(0)
# kindCd

# 학습데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
dog_data.drop(['kindNum'], axis='columns', inplace=True)
dog_data = pd.concat([dog_data, kindCd], axis=1)


# kindNum을 원핫 인코딩
kindCd_train = pd.concat((pd.get_dummies(dog_train.kindNum, columns=kindCd_train), pd.DataFrame(columns=kindCd_train))).fillna(0)

# 테스트데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
dog_train.drop(['kindNum'], axis='columns', inplace=True)
dog_train = pd.concat([dog_train, kindCd_train], axis=1)
dog_data
# dog_train

,neuterYn,sexCd,weight,noticeDays,age2,processState,1.0,2.0,3.0,4.0,...,202.0,203.0,204.0,205.0,206.0,207.0,208.0,209.0,210.0,211.0
0,N,F,7.46,10,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,N,M,7.00,14,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,U,M,4.50,11,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,N,M,10.00,8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,N,M,6.00,8,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22787,N,M,1.00,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22788,N,M,1.00,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22789,N,M,1.00,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22790,U,F,6.00,12,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis = 1)
        data = data.drop(column, axis = 1)
    return data

In [32]:
dummy_columns = ["neuterYn", "sexCd"]
data = dummy_data(dog_data, dummy_columns)
train_data = dummy_data(dog_train, dummy_columns)
data.head()
train_data.head()
# print(type(dog_data))
# print(type(data))

print(data)

data = np.array(data, dtype = np.float64)
train_data = np.array(train_data, dtype = np.float64)

# data
# print(data.shape)
# print(len(train_data))
# print(train_data.shape)

       weight  noticeDays  age2  processState  1.0  2.0  3.0  4.0  5.0  6.0  \
0        7.46          10    12             0    0    0    0    0    0    0   
1        7.00          14     1             1    0    0    0    0    0    0   
2        4.50          11     2             0    0    0    0    0    0    0   
3       10.00           8     1             0    0    0    0    0    0    0   
4        6.00           8     4             0    0    0    0    0    0    0   
...       ...         ...   ...           ...  ...  ...  ...  ...  ...  ...   
22787    1.00          10     0             1    0    0    0    0    0    0   
22788    1.00          10     0             1    0    0    0    0    0    0   
22789    1.00          10     0             1    0    0    0    0    0    0   
22790    6.00          12     3             0    0    0    0    0    0    0   
22791    3.50          10     0             0    0    0    0    0    0    0   

       ...  208.0  209.0  210.0  211.0  neuterYn_N 

In [33]:
a = data[:, :3]
b = data[:, 4:]

# print(a[0,:])
# print(b)

#numpy 배열에서 데이터 변화요인(kindCd, neuterYn, sexCd, weight, noticeDays, age2)으로 사용할 데이터를 뽑아냅니다.
xData = np.concatenate([a, b], axis = 1)

print(xData)
type(xData)
xData.shape

[[ 7.46 10.   12.   ...  1.    0.    0.  ]
 [ 7.   14.    1.   ...  0.    1.    0.  ]
 [ 4.5  11.    2.   ...  0.    1.    0.  ]
 ...
 [ 1.   10.    0.   ...  0.    1.    0.  ]
 [ 6.   12.    3.   ...  1.    0.    0.  ]
 [ 3.5  10.    0.   ...  0.    1.    0.  ]]


(22777, 186)

In [34]:
#numpy배열에서 결과(입양여부)로 사용할 데이터를 뽑아냅니다.
yData=data[:,[3]]
# yData = yData.astype('int32')
print(yData)
type(yData)
yData.dtype

[[0.]
 [1.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


dtype('float64')

In [35]:
# #뽑아낸 데이터를 tensorflow로 처리하기 위해 placeholder를 만듭니다.
X=tf.placeholder(tf.float64, [None, 186])
Y=tf.placeholder(tf.float64, [None, 1])
# Z=tf.placeholder(tf.float64)
# #다변인 선형 회귀 모델의 기울기와 y절편을 임의의 값으로 초기화합니다.
a=tf.Variable(tf.random_normal([186, 1], dtype=tf.float64))#6행 1열 난수를 발생시킵니다.
b=tf.Variable(tf.random_normal([1], dtype=tf.float64)) #1행 1열 난수를 발생시킵니다.
print(X)
print(Y)
# print(a)

Tensor("Placeholder_4:0", shape=(?, 186), dtype=float64)
Tensor("Placeholder_5:0", shape=(?, 1), dtype=float64)


In [36]:
# #행렬의 곱셈 연산을 이용해 다변인 성형 회귀 모델의 가설 식을 세웁니다. ->예측값
# y=tf.matmul(X, a)+b
# #y=tf.clip_by_value(y, 1e-5, 1-(1e-5))

# #오차 함수를 만듭니다.
# loss= tf.reduce_mean(tf.square(y-Y))
# #loss= tf.clip_by_value(loss, 1e-5, 1-(1e-5))

# #경사하강법 알고리즘을 사용해서 오차 함수 결과를 최소로 하는 식을 만듭니다. 
# gradient_descent=tf.train.GradientDescentOptimizer(0.1).minimize(loss)


# # 시그모이드 방정식을 만듭니다.
y = tf.sigmoid(tf.matmul(X, a) + b)
# # 오차 함수
# loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y))
# 경사 하강 알고리즘
# # gradient_descent = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

# gradient_descent = tf.train.GradientDescentOptimizer(learning_rate=1e-2).minimize(loss)

# y=tf.nn.softmax(tf.matmul(X, a)+ b)
# cost= tf.reduce_mean(-tf.reduce_sum(Y*tf.log(y), reduction_indices=1))
cost = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y))
optimizer=tf.train.GradientDescentOptimizer(0.01).minimize(cost)

predicted = tf.cast(y >= 0.5, dtype = tf.float64)

# 예상값이 실제값과 일치하는 정도(정확도)를 계산합니다.
# tf.equal(): tensorflow에서 인수로 지정된 값이 같으면 True, 다르면 False 결과를 tf.float64로 캐스팅 시키고 시행된 전체 평균을 계산합니다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float64))  # 정확도 (0 또는 1) 

with tf.Session() as sess:  # 세션 구간을 만듦
    sess.run(tf.global_variables_initializer())
    for i in range(10001):
        # count : 학습 횟수, a1 : 기울기(공부 시간), asess.run(cost, feed_dict={X:xData, Y:yData}))
        a_, b_, cost_, _ = sess.run([a, b, cost, optimizer], feed_dict = {X:xData, Y:yData})
        if i % 500 == 0:
            print(sess.run(y, feed_dict={X:xData}))
            print('count:{}, b = {}, loss = {}'.format(i, b_, cost_)) # 학습시킨 데이터 출력
    av=sess.run([accuracy], feed_dict={X:xData, Y:yData})
    print("정확도: {}".format(av))
                
    for i in range(len(train_data)):
        new_x=train_data[i, :].reshape(1,186)
        result, new_y = sess.run([predicted, y], feed_dict={X:new_x})
        print('입양 여부 : %s,    입양 확률 : %8.4f%%' % ('입양' if result == 1 else '안락사', new_y*100))
    
#     pv, y, av=sess.run([predicted, y, accuracy], feed_dict={X:xData, Y:yData})
#     print("정확도: {}".format(av))
    
    saver = tf.train.Saver()
    save_path = saver.save(sess, './dog.ckpt')
    print('학습된 모델을 저장했습니다')

[[0.99999986]
 [0.01182814]
 [0.13831727]
 ...
 [0.01707027]
 [0.28681039]
 [0.01708328]]
count:0, b = [-1.12012987], loss = 1.9829673927170584
[[0.84942395]
 [0.55889671]
 [0.49480591]
 ...
 [0.61306176]
 [0.57911461]
 [0.58830469]]
count:500, b = [-1.07622445], loss = 0.7554424217618166
[[0.78271259]
 [0.53660201]
 [0.46158095]
 ...
 [0.56610793]
 [0.67999402]
 [0.54563783]]
count:1000, b = [-1.06096327], loss = 0.7154396409334858
[[0.77835953]
 [0.51639836]
 [0.43802232]
 ...
 [0.52826507]
 [0.7478362 ]
 [0.50871371]]
count:1500, b = [-1.03995408], loss = 0.691300702644928
[[0.77296159]
 [0.50353784]
 [0.41818199]
 ...
 [0.50344231]
 [0.78834229]
 [0.48443486]]
count:2000, b = [-1.01464658], loss = 0.6755306648766427
[[0.76634081]
 [0.49467086]
 [0.40153703]
 ...
 [0.48691737]
 [0.81335416]
 [0.46821664]]
count:2500, b = [-0.98633102], loss = 0.6647771798793645
[[0.75887056]
 [0.4878935 ]
 [0.38763206]
 ...
 [0.47566778]
 [0.8292858 ]
 [0.4571246 ]]
count:3000, b = [-0.95600156], lo

입양 여부 : 입양,    입양 확률 :  70.1662%
입양 여부 : 입양,    입양 확률 :  67.0559%
입양 여부 : 안락사,    입양 확률 :  43.2550%
입양 여부 : 입양,    입양 확률 :  91.1034%
입양 여부 : 입양,    입양 확률 :  76.3041%
입양 여부 : 입양,    입양 확률 :  61.1621%
입양 여부 : 입양,    입양 확률 :  53.1492%
입양 여부 : 안락사,    입양 확률 :  35.7528%
입양 여부 : 입양,    입양 확률 :  51.7469%
입양 여부 : 안락사,    입양 확률 :  34.4755%
입양 여부 : 입양,    입양 확률 :  53.2073%
입양 여부 : 입양,    입양 확률 :  75.0748%
입양 여부 : 입양,    입양 확률 :  89.3841%
입양 여부 : 안락사,    입양 확률 :  36.3447%
입양 여부 : 입양,    입양 확률 :  51.7418%
입양 여부 : 입양,    입양 확률 :  82.9589%
입양 여부 : 안락사,    입양 확률 :  43.4224%
입양 여부 : 입양,    입양 확률 :  71.2109%
입양 여부 : 안락사,    입양 확률 :  33.4738%
입양 여부 : 안락사,    입양 확률 :  39.4474%
입양 여부 : 안락사,    입양 확률 :  39.1476%
입양 여부 : 안락사,    입양 확률 :  37.8059%
입양 여부 : 입양,    입양 확률 :  73.3752%
입양 여부 : 안락사,    입양 확률 :  35.6633%
입양 여부 : 안락사,    입양 확률 :  40.2381%
입양 여부 : 안락사,    입양 확률 :  39.6352%
입양 여부 : 안락사,    입양 확률 :  40.1640%
입양 여부 : 입양,    입양 확률 :  68.6759%
입양 여부 : 안락사,    입양 확률 :  37.6133%
입양 여부 : 안락사,    입양 확률 :  38.1

입양 여부 : 안락사,    입양 확률 :  28.2359%
입양 여부 : 안락사,    입양 확률 :  26.8746%
입양 여부 : 안락사,    입양 확률 :  23.1323%
입양 여부 : 입양,    입양 확률 :  87.2060%
입양 여부 : 안락사,    입양 확률 :  21.3740%
입양 여부 : 안락사,    입양 확률 :  31.6708%
입양 여부 : 안락사,    입양 확률 :  14.1453%
입양 여부 : 안락사,    입양 확률 :  39.7596%
입양 여부 : 안락사,    입양 확률 :  40.8647%
입양 여부 : 안락사,    입양 확률 :  34.8700%
입양 여부 : 입양,    입양 확률 :  90.3874%
입양 여부 : 안락사,    입양 확률 :  44.7373%
입양 여부 : 안락사,    입양 확률 :  44.7373%
입양 여부 : 안락사,    입양 확률 :  45.0496%
입양 여부 : 입양,    입양 확률 :  74.5856%
입양 여부 : 입양,    입양 확률 :  79.9920%
입양 여부 : 안락사,    입양 확률 :  33.1880%
입양 여부 : 입양,    입양 확률 :  88.0310%
입양 여부 : 입양,    입양 확률 :  69.7467%
입양 여부 : 입양,    입양 확률 :  70.4693%
입양 여부 : 안락사,    입양 확률 :  45.6576%
입양 여부 : 입양,    입양 확률 :  88.7662%
입양 여부 : 안락사,    입양 확률 :  34.7790%
입양 여부 : 안락사,    입양 확률 :  34.7790%
입양 여부 : 안락사,    입양 확률 :  35.5273%
입양 여부 : 안락사,    입양 확률 :  28.3426%
입양 여부 : 안락사,    입양 확률 :  19.2214%
입양 여부 : 안락사,    입양 확률 :  39.5094%
입양 여부 : 안락사,    입양 확률 :  39.4357%
입양 여부 : 입양,    입양 확률 :